In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# 1) design model (input, output size, forward pass)
# 2) construct loss and optimizer
# 3) Training loop
# -forward pass: compute prediction
# -backward pass: gradients
# -update weights

In [6]:
#prepare data
bc = datasets.load_breast_cancer()
X,y= bc.data,bc.target

n_samples, n_features=X.shape
print(X.shape)

(569, 30)


In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=1234)

In [18]:
#scale
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [20]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [21]:
#model
#f=wx+b, sigmoid at the end
class logisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(logisticRegression,self).__init__()
        self.linear=nn.Linear(n_input_features,1)

    def forward(self,x):
        ypred=torch.sigmoid(self.linear(x))
        return ypred




model= logisticRegression(n_features)


In [22]:
#loss and optimizer
learning_rate=0.01
criterion=nn.BCELoss()
optimizer= torch.optim.SGD(model.parameters(),lr=learning_rate)

In [23]:
num_epoch =100
for epoch in range(num_epoch):
    #forwardpass
    ypred=model(X_train)
    loss=criterion(ypred,y_train)

    #backwardpass
    loss.backward()#dl/dw


    optimizer.step()

    optimizer.zero_grad()
    if (epoch+1) % 10 ==0:
        print(f'epoch {epoch+1}: loss = {loss.item():.3f}')





epoch 10: loss = 0.608
epoch 20: loss = 0.493
epoch 30: loss = 0.423
epoch 40: loss = 0.376
epoch 50: loss = 0.341
epoch 60: loss = 0.314
epoch 70: loss = 0.293
epoch 80: loss = 0.275
epoch 90: loss = 0.261
epoch 100: loss = 0.248


In [24]:
with torch.no_grad():
    ypred =model(X_test)
    ypred_cls=ypred.round()#if value is greater than 0.5 it will be rounded to 1 other vise zero

    acc=ypred_cls.eq(y_test).sum()/float(y_test.shape[0])
    #for every ypred taht is equale to y_test sum it up
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9123
